In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import warnings
import sys
import gc
import torch
warnings.filterwarnings("ignore")
base_dir = os.path.split(os.getcwd())[0]
sys.path.append(base_dir)

from pyfunctions.local_importance import *
from pyfunctions._integrated_gradients import configure_interpretable_embeddings, remove_interpretable_embeddings

In [77]:
# set up env
#!python3.12 -m pip install lime
#!python3.12 -m pip install shap
#!python3.12 -m pip install tf-keras==2.16.0
#!python3.12 -m pip install captum

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.7 MB/s eta 0:00:00:00:01


In [3]:
torch.autograd.set_grad_enabled(False)

In [4]:
device = 'cuda:0'
data_name = 'pathology' #"pathology", "sst2", "agnews", "ioi"
model_type = 'bert' #"bert", "pubmed_bert", "gpt"

In [5]:
# load in data and model
data, le_dict, tokenizer, model = load_data_and_model(data_name, model_type, device)

Token indices sequence length is longer than the specified maximum sequence length for this model (1345 > 512). Running this sequence through the model will result in indexing errors


Processing train data...


2024-09-12 12:33:14.852858: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-12 12:33:17.804137: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Processing val data...
Processing test data...
Loading in model...


In [14]:
method = "CDT" #"CDT", "lime", "shap", "IG"
sample_id = 80
text = data['docs'][sample_id]
label = data['labels'][sample_id]

In [15]:
# clear cache
gc.collect()
torch.cuda.empty_cache()

if method == "IG":
    # Remove interpratable embedding layer used by ig attribution
    remove_interpretable_embeddings(model,interpretable_embedding1,
                                  interpretable_embedding2,
                                  interpretable_embedding3)

In [16]:
# passed: CDT, LIME, shap, IG

if method == "IG":
    if not type(model.get_input_embeddings()).__name__ == "InterpretableEmbeddingBase":
        interpretable_embedding1, interpretable_embedding2, interpretable_embedding3 = configure_interpretable_embeddings(model)
        

IG_interpretable_embeds = (interpretable_embedding1, interpretable_embedding2, interpretable_embedding3)
scores = run_local_importance(text, label, model, tokenizer, le_dict, device, max_seq_len=512, method=method, class_names=["3", "4", "5"],
                              IG_interpretable_embeds=IG_interpretable_embeds)

if method == "IG":
    # Remove interpratable embedding layer used by ig attribution
    remove_interpretable_embeddings(model,interpretable_embedding1,
                                  interpretable_embedding2,
                                  interpretable_embedding3)

Viz rel: 


Viz irrel: 


Viz rel-irrel: 
